<a href="https://www.kaggle.com/code/yoanndev90/open-insect-id-notebook?scriptVersionId=297309511" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
pip install ijson

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 4.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
import glob
import re
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
from collections import Counter, defaultdict
import ijson
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

In [3]:
import sys
sys.path.append('/kaggle/working')
from utils import *

imported_funcs = [name for name in dir() if not name.startswith('_') and callable(globals()[name])]
print("Fonctions importées :", imported_funcs)

Fonctions importées : ['Counter', 'DataLoader', 'Dataset', 'Path', 'ThreadPoolExecutor', 'as_completed', 'build_hierarchy_labels', 'build_taxa_maps', 'defaultdict', 'exit', 'get_ipython', 'parse_taxonomy', 'parse_taxonomy_folders', 'quit', 'save_hierarchy_map', 'scan_corrupted_images', 'tree', 'verify_image_validity']


In [27]:
from utils import parse_taxonomy, verify_image_validity

example = "train/train/00980_Animalia_Arthropoda_Insecta_Lepidoptera_Erebidae_Arctia_virginalis/464f3a34-4c04-4eb3-afa2-6cb7444c3fa3.jpg"
taxonomy = parse_taxonomy(example)
print("Résultat:", taxonomy)
validity = verify_image_validity(example)
print("Image valide" if validity else "Image invalide")

Résultat: {'ordre': 'Lepidoptera', 'famille': 'Erebidae', 'genre': 'Arctia', 'espece': 'virginalis'}
Image valide


In [6]:
from utils import tree

# total = tree(''/kaggle/input/inaturalist-insects/'', 2)                   # Très long temps d'attente (30min-1h)
total = tree('/kaggle/input/inaturalist-insects/val-json/', 2)
print(f"\nNombre total de fichiers : {total}")


Nombre total de fichiers : 1


In [8]:
import ijson

def load_annotated_images(json_path):
    """Charge lat/lon par file_name depuis train_mini_json ou train_json."""
    annotated = {}
    with open(json_path, 'rb') as f:
        parser = ijson.items(f, 'images.item')
        for img in parser:
            filename = img.get('file_name', '')
            lat = float(img.get('latitude')) if img.get('latitude') is not None else 0.0
            lon = float(img.get('longitude')) if img.get('longitude') is not None else 0.0
            annotated[filename] = (lat, lon)
    return annotated

In [9]:
from collections import Counter

def compute_stats(full_taxa_map, full_geo_db, species_encountered):
    """Calcule toutes stats."""
    ordre_count = Counter()
    famille_count = Counter()
    genre_count = Counter()
    espece_count = Counter()
    
    for taxon_key, hier in full_taxa_map.items():
        ordre_count[hier['ordre']] += 1
        famille_count[hier['famille']] += 1
        genre_count[hier['genre']] += 1
        espece_count[hier['espece']] += 1
    
    homonyms_count = len([s for s in species_encountered if len(species_encountered[s]) > 1])
    homonyms_dirs = sum(len(species_encountered[s]) for s in species_encountered if len(species_encountered[s]) > 1)
    
    taxon_geo_counts = {str(k): len(v) for k, v in full_geo_db.items()}
    geo_taxa = len(full_geo_db)
    total_taxa = len(full_taxa_map)
    multi_geo = sum(1 for c in taxon_geo_counts.values() if c > 1)
    
    return {
        'total_dirs': len(species_encountered),
        'unique_taxa': total_taxa,
        'geo_coverage': geo_taxa / total_taxa if total_taxa else 0,
        'homonyms': {'names': homonyms_count, 'dirs': homonyms_dirs},
        'hierarchy': {
            'ordres': len(ordre_count), 'familles': len(famille_count), 'genres': len(genre_count)
        },
        'taxon_geo_counts': taxon_geo_counts
    }, ordre_count, famille_count, genre_count

In [10]:
from utils import parse_taxonomy_folders, build_taxa_maps

annotated_images = load_annotated_images('/kaggle/input/inaturalist-insects/train-json/train.json')
species_encountered, unparsed_dirs = parse_taxonomy_folders('/kaggle/input/inaturalist-insects/train/train/')
full_taxa_map, full_geo_db = build_taxa_maps(species_encountered, annotated_images, '/kaggle/input/inaturalist-insects/train/train/')
stats, ordre_count, famille_count, genre_count = compute_stats(full_taxa_map, full_geo_db, species_encountered)

print(f"✅ {stats['unique_taxa']} taxons (sur {stats['total_dirs']} dossiers)")
print(f"Homonymes: {stats['homonyms']['names']} noms => {stats['homonyms']['dirs']} dossiers")
print(f"Hiérarchie: {stats['hierarchy']['ordres']} ordres, {stats['hierarchy']['familles']} familles, {stats['hierarchy']['genres']} genres")
print(f"Géo: {len(full_geo_db)}/{len(full_taxa_map)} ({stats['geo_coverage']*100:.1f}%)")
print(f"Non parsés: {len(unparsed_dirs)}")


save_hierarchy_map(full_taxa_map, full_geo_db, stats, 'hierarchy_map.json')

✅ 2526 taxons (sur 2252 dossiers)
Homonymes: 185 noms => 459 dossiers
Hiérarchie: 17 ordres, 190 familles, 1472 genres
Géo: 0/2526 (0.0%)
Non parsés: 0
💾 Sauvegardé: 2526 taxons dans hierarchy_map.json


In [4]:
from utils import build_hierarchy_labels

data_dir = '/kaggle/input/inaturalist-insects/'
hierarchy_map_file = '/kaggle/working/hierarchy_map.json'
final_hierarchy = build_hierarchy_labels(data_dir, hierarchy_map_file)

print("\nExemples:")
for idx in range(5):
    labels = final_hierarchy.get(idx)
    print(f"Class {idx}: {labels}") # [ordre_id, famille_id, genre_id, espece_id]

Classes: 2526 (scan /kaggle/input/inaturalist-insects/train_mini/train_mini)
Hiérarchie: 17 ordres, 190 fam., 1472 genres
✅ 2526/2526 mappées
💾 hierarchy_labels.json prêt pour training

Exemples:
Class 0: [0, 21, 124, 0]
Class 1: [0, 21, 1048, 1]
Class 2: [0, 21, 1219, 2]
Class 3: [0, 23, 185, 3]
Class 4: [0, 23, 1090, 4]


In [5]:
class RobustImageFolder(Dataset):
    """ImageFolder skip corrompus."""
    def __init__(self, root, transform=None, corrupt_files=None):
        self.root = root
        self.transform = transform
        self.corrupt_files = set(corrupt_files or [])
        
        self.classes, self.class_to_idx = self.find_classes(self.root)
        self.samples = self.make_dataset(self.root, self.class_to_idx)
        
        self.valid_indices = []
        for i, (path, _) in enumerate(self.samples):
            if os.path.relpath(path, self.root) not in self.corrupt_files:
                self.valid_indices.append(i)
    
    def find_classes(self, directory):
        """Trouve classes (dossiers)."""
        classes = [d.name for d in os.scandir(directory) if d.is_dir()]
        classes.sort()
        class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        return classes, class_to_idx
    
    def make_dataset(self, directory, class_to_idx):
        """Construit samples comme ImageFolder."""
        samples = []
        for target_class in sorted(self.class_to_idx.keys()):
            class_index = self.class_to_idx[target_class]
            target_dir = os.path.join(directory, target_class)
            for root, _, fnames in sorted(os.walk(target_dir, followlinks=True)):
                for fname in sorted(fnames):
                    path = os.path.join(root, fname)
                    item = (path, class_index)
                    samples.append(item)
        return samples
    
    def __getitem__(self, index):
        path, target = self.samples[self.valid_indices[index]]
        img = Image.open(path).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img, target
    
    def __len__(self):
        return len(self.valid_indices)

In [6]:
from utils import scan_corrupted_images

corrupted_train, train_log = scan_corrupted_images('/kaggle/input/inaturalist-insects/train/train', max_workers=4)
corrupted_val, val_log = scan_corrupted_images('/kaggle/input/inaturalist-insects/val/val', max_workers=4)
corrupted_train_mini, train_mini_log = scan_corrupted_images('/kaggle/input/inaturalist-insects/train_mini/train_mini', max_workers=4)
corrupted_test, test_log = scan_corrupted_images('/kaggle/input/inaturalist-insects/public_test/public_test', max_workers=4)

with open(train_log) as f:
    print("\nCORROMPUS TRAIN:\n", f.read())

Scan 663682 JPG dans /kaggle/input/inaturalist-insects/train/train

TYPES D'ERREURS:
✅ 0/663682 (0.0%) en 2424.3s
Scan 25260 JPG dans /kaggle/input/inaturalist-insects/val/val

TYPES D'ERREURS:
✅ 0/25260 (0.0%) en 100.1s
Scan 126300 JPG dans /kaggle/input/inaturalist-insects/train_mini/train_mini
❌ 01493_Animalia_Arthropoda_Insecta_Lepidoptera_Lyca: OSError: image file is truncated (115 bytes not processed)
❌ 01493_Animalia_Arthropoda_Insecta_Lepidoptera_Lyca: OSError: image file is truncated (43 bytes not processed)
❌ 01493_Animalia_Arthropoda_Insecta_Lepidoptera_Lyca: UnidentifiedImageError: cannot identify image file '/kaggle/input/inatural
❌ 01493_Animalia_Arthropoda_Insecta_Lepidoptera_Lyca: UnidentifiedImageError: cannot identify image file '/kaggle/input/inatural
❌ 01493_Animalia_Arthropoda_Insecta_Lepidoptera_Lyca: OSError: image file is truncated (124 bytes not processed)
❌ 00620_Animalia_Arthropoda_Insecta_Hemiptera_Pentat: OSError: image file is truncated (62 bytes not proce

In [7]:
corrupt_train = []
with open('/kaggle/working/corrupted_train.txt') as f:
    for line in f:
        if line.strip() and not line.startswith('#'):
            corrupt_train.append(line.strip())

corrupt_val = []
with open('/kaggle/working/corrupted_val.txt') as f:
    for line in f:
        if line.strip() and not line.startswith('#'):
            corrupt_val.append(line.strip())

In [8]:
class HierarchicalInsectDataset(Dataset):
    def __init__(self, root_dir, hierarchy_map, transform=None, corrupt_files=None):
        self.root_dir = root_dir
        self.hierarchy_map = hierarchy_map
        self.transform = transform
        
        self.inner_dataset = datasets.ImageFolder(root_dir)
        
        self.valid_indices = []
        for i in range(len(self.inner_dataset)):
            class_idx = self.inner_dataset.targets[i]
            if class_idx in self.hierarchy_map:
                # Skip si basename dans corrupt_files
                path = self.inner_dataset.samples[i][0]
                if os.path.basename(path) not in (corrupt_files or []):
                    self.valid_indices.append(i)
        
        print(f"Dataset {root_dir}: {len(self.inner_dataset)} → {len(self.valid_indices)} valides")
    
    def __getitem__(self, idx):
        real_idx = self.valid_indices[idx]
        img, class_idx = self.inner_dataset[real_idx]
        
        # Labels hiérarchiques [ordre_id, famille_id, genre_id, espece_id]
        hier_labels = torch.tensor(self.hierarchy_map[class_idx])
        
        if self.transform:
            img = self.transform(img)
        
        return img, hier_labels
    
    def __len__(self):
        return len(self.valid_indices)

In [9]:
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch

train_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = HierarchicalInsectDataset(
    '/kaggle/input/inaturalist-insects/train/train', 
    final_hierarchy, 
    transform=train_transforms,
    corrupt_files=[os.path.basename(p) for p in corrupt_train]
)

val_dataset = HierarchicalInsectDataset(
    '/kaggle/input/inaturalist-insects/val/val', 
    final_hierarchy, 
    transform=val_transforms,
    corrupt_files=[os.path.basename(p) for p in corrupt_val]
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Datasets hiérarchiques:")
print(f"  Train: {len(train_dataset)} images, 2526 classes")
print(f"  Val: {len(val_dataset)} images")
print(f"  Labels: ordre/famille/genre/espece [device={device}]")

img, labels = next(iter(train_loader))
print(f"Batch shape: {img.shape}, labels shape: {labels.shape}")  # [32,3,224,224], [32,4]

Dataset /kaggle/input/inaturalist-insects/train/train: 663682 → 663682 valides
Dataset /kaggle/input/inaturalist-insects/val/val: 25260 → 25260 valides
✅ Datasets hiérarchiques:
  Train: 663682 images, 2526 classes
  Val: 25260 images
  Labels: ordre/famille/genre/espece [device=cuda]
Batch shape: torch.Size([32, 3, 224, 224]), labels shape: torch.Size([32, 4])


In [10]:
class HierarchicalMobileNetV3(nn.Module):
    """MobileNetV3 avec outputs séparés par niveau."""
    def __init__(self, num_ordre=17, num_famille=190, num_genre=1472, num_espece=2526):
        super().__init__()
        backbone = models.mobilenet_v3_large(weights='IMAGENET1K_V1')
        self.features = backbone.features
        self.avgpool = backbone.avgpool
        
        self.fc_shared = nn.Sequential(
            nn.Linear(960, 512),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3)
        )
        
        # Heads indépendants (FIX: pas cumulatif)
        self.head_ordre = nn.Linear(512, num_ordre)
        self.head_famille = nn.Linear(512, num_famille)
        self.head_genre = nn.Linear(512, num_genre)
        self.head_espece = nn.Linear(512, num_espece)
    
    def forward(self, x, return_probs=False):
        x = self.features(x)
        x = self.avgpool(x)
        feats = torch.flatten(x, 1)
        shared = self.fc_shared(feats)
        
        ordre = self.head_ordre(shared)
        famille = self.head_famille(shared)
        genre = self.head_genre(shared)
        espece = self.head_espece(shared)
        
        if return_probs:
            return (F.softmax(ordre, dim=1), F.softmax(famille, dim=1), 
                   F.softmax(genre, dim=1), F.softmax(espece, dim=1))
        
        max_classes = 2526
        preds = torch.zeros(x.size(0), 4, max_classes, device=x.device)
        preds[:, 0, :ordre.size(1)] = ordre
        preds[:, 1, :famille.size(1)] = famille
        preds[:, 2, :genre.size(1)] = genre
        preds[:, 3, :espece.size(1)] = espece
        
        return preds  # [B,4,2526]

In [11]:
with open('hierarchy_labels.json') as f:
    stats = json.load(f)['stats']
    
num_ordre = stats['ordres']      # 17
num_famille = stats['familles']  # 190
num_genre = stats['genres']     # 1472
num_espece = stats['total_classes']  # 2526

In [12]:
class HierarchicalLoss(nn.Module):
    def __init__(self, num_classes_per_level):
        super().__init__()
        self.ce = nn.CrossEntropyLoss()
        self.weights = torch.tensor([1.0, 2.0, 5.0, 10.0])
        self.num_classes = num_classes_per_level  # [17,190,1472,2526]
    
    def forward(self, preds, targets):
        loss = 0
        for i in range(4):
            mask = torch.arange(self.num_classes[i], device=preds.device)
            lvl_pred = preds[:, i, mask]
            lvl_loss = self.ce(lvl_pred, targets[:, i])
            loss += self.weights[i] * lvl_loss
        return loss / self.weights.sum()

criterion = HierarchicalLoss([num_ordre, num_famille, num_genre, num_espece])

In [13]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

model = HierarchicalMobileNetV3(num_ordre, num_famille, num_genre, num_espece).to(device)

print(f"Model: {sum(p.numel() for p in model.parameters()):,} params")
print(f"✅ GPU: {next(model.parameters()).device}")

model.eval()
with torch.no_grad():
    batch_img, batch_labels = next(iter(train_loader))
    preds = model(batch_img.to(device))  # [32,4,2526]
    loss = criterion(preds, batch_labels.to(device))
    print(f"✅ Test OK: preds={preds.shape}, loss={loss.item():.3f}")

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth


100%|██████████| 21.1M/21.1M [00:00<00:00, 181MB/s]


Model: 5,622,173 params
✅ GPU: cuda:0
✅ Test OK: preds=torch.Size([32, 4, 2526]), loss=7.121


In [14]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

In [6]:
import os

os.makedirs("onnx", exist_ok=True)

In [15]:
import torch
from torch.amp import autocast, GradScaler
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch.onnx


def setup_training(
    model,
    num_ordre,
    num_famille,
    num_genre,
    num_espece,
    checkpoint_path=None,
    device=None,
    lr=1e-4,
    weight_decay=1e-4,
    patience=10,
):
    """
    Prépare l'entraînement hiérarchique, avec tentative de reprise depuis un checkpoint.

    - Si `checkpoint_path` existe et se charge correctement -> reprise.
    - Sinon -> entraînement à partir de zéro.
    """

    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    criterion = HierarchicalLoss([num_ordre, num_famille, num_genre, num_espece])
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, patience=3, factor=0.5, min_lr=1e-6
    )
    scaler = GradScaler('cuda' if torch.cuda.is_available() else 'cpu')

    # Valeurs par défaut (from scratch)
    start_epoch = 0
    best_val_acc = 0.0
    patience_counter = 0

    train_losses, val_losses, val_accs = [], [], []
    hierarchy_accs = {"ordre": [], "famille": [], "genre": [], "espece": []}

    # Tentative de reprise
    if checkpoint_path is not None:
        try:
            checkpoint = torch.load(checkpoint_path, map_location=device)
            model.load_state_dict(checkpoint['model'])

            if 'optim' in checkpoint:
                optimizer.load_state_dict(checkpoint['optim'])
            if 'scheduler' in checkpoint:
                scheduler.load_state_dict(checkpoint['scheduler'])

            start_epoch = checkpoint.get('epoch', -1) + 1
            best_val_acc = checkpoint.get('val_acc', 0.0)
            stats = checkpoint.get('stats', {})
            train_losses = stats.get('train_losses', [])
            val_losses = stats.get('val_losses', [])
            val_accs = stats.get('val_accs', [])

            if 'hierarchy_accs' in stats:
                for k in hierarchy_accs.keys():
                    if k in stats['hierarchy_accs']:
                        hierarchy_accs[k] = list(stats['hierarchy_accs'][k])

            print(f"✅ Reprise depuis '{checkpoint_path}' à l'epoch {start_epoch}, Best Acc: {best_val_acc:.4f}")
            print(f"📊 History: {len(train_losses)} epochs déjà faits")
        except FileNotFoundError:
            print(f"⚠️ Checkpoint '{checkpoint_path}' introuvable, entraînement à partir de zéro.")
        except Exception as e:
            print(f"⚠️ Impossible de charger le checkpoint '{checkpoint_path}' ({e}), entraînement à partir de zéro.")

    print("🚀 Début entraînement hiérarchique")

    return {
        "model": model,
        "device": device,
        "criterion": criterion,
        "optimizer": optimizer,
        "scheduler": scheduler,
        "scaler": scaler,
        "start_epoch": start_epoch,
        "best_val_acc": best_val_acc,
        "patience": patience,
        "patience_counter": patience_counter,
        "train_losses": train_losses,
        "val_losses": val_losses,
        "val_accs": val_accs,
        "hierarchy_accs": hierarchy_accs,
    }

In [ ]:
import torch
from torch.amp import autocast, GradScaler
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch.onnx

print(f"Labels shape: {labels.shape}, dtype: {labels.dtype}")
print(f"Sample labels[0]: {labels[0]}")
print(f"preds shape: {preds.shape}")

state = setup_training(
    model,
    num_ordre, num_famille, num_genre, num_espece,
    checkpoint_path="/kaggle/working/best_hier_model.pth"
)

model = state["model"]
device = state["device"]
criterion = state["criterion"]
optimizer = state["optimizer"]
scheduler = state["scheduler"]
scaler = state["scaler"]
start_epoch = state["start_epoch"]
best_val_acc = state["best_val_acc"]
patience = state["patience"]
patience_counter = state["patience_counter"]
train_losses = state["train_losses"]
val_losses = state["val_losses"]
val_accs = state["val_accs"]
hierarchy_accs = state["hierarchy_accs"]

for epoch in range(start_epoch, 50):
    # === TRAIN ===
    model.train()
    train_loss = 0
    pbar = tqdm(train_loader, desc=f'Epoch {epoch:2d}')
    
    for batch_idx, (images, labels) in enumerate(pbar):
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        
        optimizer.zero_grad(set_to_none=True)
        
        with autocast('cuda'):
            preds = model(images)  # [B,4,2526]
            loss = criterion(preds, labels)
        
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()
        pbar.set_postfix(loss=f"{loss.item():.4f}")
    
    avg_train_loss = train_loss / len(train_loader)
    
    # === VAL ===
    model.eval()
    val_loss_total = 0
    val_correct = [0,0,0,0]  # [ordre, famille, genre, espèce]
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            preds = model(images)
            loss = criterion(preds, labels)
            val_loss_total += loss.item()
        
            for lvl in range(4):
                logits = preds[:, lvl, :criterion.num_classes[lvl]]
                lvl_preds = logits.argmax(1)
                val_correct[lvl] += (lvl_preds == labels[:, lvl]).sum().item()
        
            val_total += labels.size(0)

    avg_val_loss = val_loss_total / len(val_loader)
    hierarchy_accs_current = {
        'ordre': val_correct[0]/val_total,
        'famille': val_correct[1]/val_total,
        'genre': val_correct[2]/val_total,
        'espece': val_correct[3]/val_total
    }
    val_acc = hierarchy_accs_current['espece']
    
    print(f"Epoch {epoch:2d}: Train={avg_train_loss:.4f} | Val={avg_val_loss:.4f}")
    print(f"  Accs - Ordre:{hierarchy_accs_current['ordre']:.4f} | Fam:{hierarchy_accs_current['famille']:.4f} | "
          f"Genre:{hierarchy_accs_current['genre']:.4f} | Esp:{val_acc:.4f}")
    
    # Update tracking lists
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    val_accs.append(val_acc)
    for k in hierarchy_accs:
        hierarchy_accs[k].append(hierarchy_accs_current[k])
    
    # PLOT avec vraies accuracies
    plt.figure(figsize=(15,4))
    plt.subplot(1,3,1); 
    plt.plot(train_losses, 'b-', label='Train'); plt.plot(val_losses, 'r-', label='Val'); plt.legend(); plt.title('Loss')
    plt.subplot(1,3,2); plt.plot(val_accs, 'g-'); plt.title('Val Espèce Acc'); plt.ylim(0,1)
    plt.subplot(1,3,3); 
    keys = ['Ordre','Famille','Genre','Espèce']
    accs = [hierarchy_accs_current['ordre'], hierarchy_accs_current['famille'], 
            hierarchy_accs_current['genre'], hierarchy_accs_current['espece']]
    plt.bar(keys, accs); plt.title('Val Hierarchy Accs'); plt.ylim(0,1)
    plt.tight_layout(); plt.show()
    
    # EXPORT ONNX toutes les 5 epochs
    if (epoch + 1) % 5 == 0:
        model.eval()
        dummy_input = torch.randn(1, 3, 224, 224).to(device)
        
        # Test forward pour voir les vraies outputs
        with torch.no_grad():
            test_output = model(dummy_input)
            print(f"Model outputs shape: {test_output.shape if isinstance(test_output, torch.Tensor) else [o.shape for o in test_output]}")
        
        # Export simple (1 input → 1 output principal)
        torch.onnx.export(
            model, 
            dummy_input, 
            f"onnx/insect_model_epoch_{epoch+1}.onnx",
            export_params=True, 
            opset_version=17,  # +récent = mieux support
            do_constant_folding=True,
            input_names=['image'],
            output_names=['logits'],  # 1 seul output
            dynamic_axes={
                'image': {0: 'batch_size'},
                'logits': {0: 'batch_size'}
            }
        )
        print(f"💾 ONNX exported: onnx/insect_model_epoch_{epoch+1}.onnx")

    
    # SAVE BEST
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        torch.save({
            'model': model.state_dict(),
            'optim': optimizer.state_dict(),
            'epoch': epoch,
            'val_acc': val_acc,
            'hierarchy_accs': hierarchy_accs_current,
            'stats': {'train_losses': train_losses.copy(), 'val_losses': val_losses.copy(), 
                     'val_accs': val_accs.copy(), 'hierarchy_accs': hierarchy_accs.copy()}
        }, 'best_hier_model.pth')
        print(f"🔥 BEST: Esp.Acc={val_acc:.4f}")
    else:
        patience_counter += 1
    
    scheduler.step(avg_val_loss)
    
    if patience_counter >= patience:
        print(f"🛑 Early Stop at epoch {epoch}")
        break

print(f"🏆 FINAL: Best Espèce Acc = {best_val_acc:.4f}")

Labels shape: torch.Size([32, 4]), dtype: torch.int64
Sample labels[0]: tensor([   7,   68, 1341,  953])
preds shape: torch.Size([32, 4, 2526])
✅ Reprise depuis '/kaggle/working/best_hier_model.pth' à l'epoch 13, Best Acc: 0.7730
📊 History: 13 epochs déjà faits
🚀 Début entraînement hiérarchique


Epoch 13:  16%|█▌        | 3237/20741 [15:50<1:27:38,  3.33it/s, loss=0.7563]